# Combinación Demanda y Clima

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Gráficos de datos
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

## Lectura y configuración del clima

In [ ]:
# Automatización: el usuario ingresa la dirección del archivo con los datos de la temperatura
weather = pd.read_csv(r'---INSERTAR .CSV DE LOS DATOS DE CLIMA DE UN MEDIDOR---', parse_dates = True)
print(f'Cantidad de datos: {weather.shape[0]}')
print('\n')
print(weather.head(10))

In [ ]:
weather.rename(columns = {'horaLocalidad':'datetime'}, inplace = True)
# La columna 'datetima' es configurada como objeto de tiempo y sus segundos llevados a cero
weather['datetime'] = pd.to_datetime(weather['datetime'])
weather['datetime'] = weather['datetime'].dt.round('min')
weather.sort_values(by=['datetime'], axis = 0, ascending = True, inplace = True)
weather.reset_index(inplace = True, drop = True)

In [ ]:
# Función para redondear los minutos a multiplos de 15
def round_minutes(dt, resolution):
    new_minute = (dt.minute // resolution + 1) * resolution
    return dt + timedelta(minutes=new_minute - dt.minute)

In [ ]:
lista = []

for date in weather['datetime']:    
    resolusion = round_minutes(date , 15)
    # date = date.replace(resolusion)
    lista.append(resolusion)
    # print(f'{fecha} con una resolusión de 15min es redondeada a: {resolusion}')
    
# print(len(lista))
weather['new_datetime'] = lista
weather.drop(columns='datetime', inplace=True)
# De datos duplicados, solo se mantiene la medición más reciente. 
weather.drop_duplicates(subset = 'new_datetime', keep = 'last', inplace = True)
weather.rename(columns = {'new_datetime':'datetime'}, inplace = True)
weather = weather.set_index('datetime')

In [ ]:
print(weather.info())
print('\n')
print(weather.index.min())
print(weather.index.max())

In [ ]:
# Datos sin filtrar
fig = go.Figure()
fig.add_trace(go.Scatter(x=weather_data.index, y=weather_data['temp'],
                         mode='lines',
                         name='Clima'))

# adjust layout
fig.update_traces(line=dict(width=0.75))
fig.show()

## Lectura y configuración de la demanda

In [ ]:
# Automatizacion: El usuario inserta la dirección del archivo con los datos de demanda
demand = pd.read_csv(r'---INSERTAR .CSV DE LOS DATOS DE DEMANDA DE UN MEDIDOR---')
print(f'Cantidad de datos: {demand.shape[0]}')
print('\n')
print(demand.head(10))

In [ ]:
demand.drop(columns = 'terminal', inplace = True)
demand.rename(columns = {'fechahora' : 'datetime', 'demanda_activa' : 'y[kW]' }, inplace = True)

In [ ]:
#Convierto a tipo DateTimeIndex la columna "Datetime"
demand['datetime'] = pd.to_datetime(demand['datetime'])
demand.sort_values(by=['datetime'], axis = 0, ascending = True, inplace = True)
demand.reset_index(inplace = True, drop = True)

In [ ]:
# De datos duplicados, solo se mantiene la medición más reciente. 
demand.drop_duplicates(subset = 'datetime', keep = 'last', inplace = True)
demand.set_index('datetime', inplace = True)

In [ ]:
# Fecha de inicio coincidente con la del clima (recorte de datos en demanda)
demand = demand.loc['2021-04-12 08:45:00':]

In [ ]:
print(demand.info())
print('\n')
print(min(demand.index))
print(max(demand.index))

In [ ]:
# Datos sin filtrar
fig = go.Figure()
fig.add_trace(go.Scatter(x=demand.index, y=demand['y[kW]'],
                         mode='lines',
                         name='Energía'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.show()

## Combinando ambos datos

In [ ]:
comb = weather.copy()
# Combino los datos agregando la columna de temperatura a 'data_comb'
comb['y[kW]'] = demand['y[kW]']
comb = comb[['y[kW]', 'temp']]

In [ ]:
comb.head()

In [ ]:
# Debido a la unión, muy probablemente existan datos NaN en alguna (o ambas) de las columnas. 
print(comb['temp'].isna().sum())
print(comb['y[kW]'].isna().sum())

In [ ]:
print(f'Verificación de la frecuencia de los datos: {comb.index.freq}')
print('En caso de no tener frecuencia, se establece la de 15MIN')

In [ ]:
# Custom range
data_range = pd.date_range(start = min(comb.index),
                          end = max(comb.index),
                          freq = '15min') 
data_range
#freq = '15min' indica frecuencia por hora.
#Explicación: genero un dataframe con una frecuencia horaria desde el valor minimo al máximo del index (tipo datetime)
#del dataframe original. 
#Con esto lo que obtengo es una serie de fechas completa. 
#Al hacer mas adelante la diferencia entre ambos dataframe, voy a obtener los "días perdidos" del dataframe original. 
# https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases

In [ ]:
#la diferencia entre ambos df indica la cantidad de valores perdidos en el df_original
print(f'La diferencia de longitud entre el rango customizado de datos y nuestro dataset es {(len(data_range)-len(comb))}')

In [ ]:
#Imprimo las fechas que faltan
print(data_range.difference(comb.index))

In [ ]:
# El siguiente comando adjunta los datos "datetime" perdidos (missing) al dataset original
# pero va a generar valores NaN para la variable Target (y[kW])
comb = comb.reindex(data_range)

print(f'La frecuencia de los datos es: {comb.index.freq}')

In [ ]:
print(f'{comb.info()}\n')
print(comb.isnull().sum())
# df1 = full_comb[full_comb.isna().any(axis=1)]
# print (df1)

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=comb.index, y=comb['temp'],
                         mode='lines', 
                         name='Clima'))

# adjust layout
fig.update_traces(line=dict(width=1.5))
fig.show()

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=comb.index,y=comb['y[kW]'],
                         mode='lines',
                         name='Energia'))

# adjust layout
fig.update_traces(line=dict(width=1.5))
fig.show()

## Interpolación

In [ ]:
# Llenamos estos valores blancos con valores que se encuentran en una curva lineal entre puntos de datos existentes
comb['temp'].interpolate(method='linear', inplace=True)

In [ ]:
# Verifico datos nulos luego de interpolar valores para la temperatura
print(comb['temp'].isnull().sum())

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=comb.index, y=comb['temp'],
                         mode='lines', 
                         name='Clima'))

# adjust layout
fig.update_traces(line=dict(width=1.5))
fig.show()

In [ ]:
# Llenamos estos valores blancos con valores que se encuentran en una curva lineal entre puntos de datos existentes
comb['y[kW]'].interpolate(method='linear', inplace=True)

In [ ]:
# Verifico datos nulos luego de interpolar valores para la energía
print(comb['y[kW]'].isnull().sum())

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=comb.index,y=comb['y[kW]'],
                         mode='lines',
                         name='Energia'))

# adjust layout
fig.update_traces(line=dict(width=1.5))
fig.show()

In [ ]:
# Verifico que la longitud de datos en ambas columnas sea igual
comb.info()

## Correlación demanda y clima

In [ ]:
actual_load_correlations = comb.corr()['y[kW]']
actual_load_correlations.sort_values(ascending=False)

# CSV

In [ ]:
comb['datetime'] = comb.index
comb.to_csv('datos_comb.csv', index = False, encoding='utf-8')